In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import pandas as pd
from sklearn.utils import shuffle

from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
### Loading Data

df_data_test = pd.read_json('/content/drive/MyDrive/thesis_files/df_data_test_31.05.json', orient='index')
df_data_train = pd.read_json('/content/drive/MyDrive/thesis_files/df_data_train_31.05.json', orient='index')

In [ ]:
### creating balanced dataset

shuffle(df_data_train).reset_index(drop=True)
shuffle(df_data_test).reset_index(drop=True)

In [ ]:
def create_balanced_dataset(df):
    # Group the DataFrame by the 'label' column
    grouped = df.groupby('label')

    # Find the smallest group size
    min_group_size = min(grouped.size())

    # Sample an equal number of rows from each group
    balanced_df = grouped.apply(lambda x: x.sample(min_group_size)).reset_index(drop=True)

    return balanced_df


df_data_train_balanced = create_balanced_dataset(df_data_train)
df_data_test_balanced = create_balanced_dataset(df_data_test)

In [ ]:
# Preparing the training and testing datasets
X_train = df_data_train['snippet']
y_train = df_data_train['label']
X_test = df_data_test['snippet']
y_test = df_data_test['label']

In [ ]:
# Preparing the training and testing datasets
#X_train = df_data_train_balanced['snippet']
#y_train = df_data_train_balanced['label']
#X_test = df_data_test_balanced['snippet']
#y_test = df_data_test_balanced['label']

NameError: ignored

In [ ]:
### Simple LSTM Classifier ###

max_len = 100
embedding_dim = 100
vocab_size = 10000

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# Padding
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# Building the model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

# Compiling
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Compute class weights based on inverse ratio
class_weights = {0: 1, 1: 1 / np.mean(y_train)}

In [ ]:
# Compute class weights based on squared root of the ratio

_, counts = np.unique(y_train, return_counts=True)
class_freq = counts / len(y_train)

# Compute class weights based on the square root of the inverse ratio
class_weights = {class_id: np.sqrt(1 / freq) for class_id, freq in enumerate(class_freq)}

class_weights[1] = class_weights[1] * 1

# Print the class weights
print(class_weights)



{0: 1.0087557179561957, 1: 7.606354545280391}


In [ ]:
### focal loss
import tensorflow as tf
from tensorflow.keras import backend as K



def focal_loss(alpha=0.25, gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        """
        Focal Loss for binary classification
        FL(p_t) = -alpha(1 - p_t)^{gamma} * log(p_t)
        where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        fl1 = - alpha * tf.pow(1. - pt_1, gamma) * tf.math.log(pt_1)
        fl0 = -(1 - alpha) * tf.pow(pt_0, gamma) * tf.math.log(1. - pt_0)

        return K.mean(fl1 + fl0)
    return focal_crossentropy



In [ ]:

### Compiling and Training with focal loss
model.compile(loss=focal_loss(alpha=0.75, gamma=2.0), optimizer='adam', metrics=['accuracy'])


### can also add class weights here
model.fit(train_padded, y_train,
                    epochs=10, batch_size=32, class_weight=class_weights,
                    validation_data=(test_padded, y_test),
                    verbose=1)


Epoch 1/10
47795/47795 [==============================] - 356s 7ms/step - loss: 0.0404 - accuracy: 0.9832 - val_loss: 0.0132 - val_accuracy: 0.9812
Epoch 2/10
47795/47795 [==============================] - 318s 7ms/step - loss: 0.0364 - accuracy: 0.9828 - val_loss: 0.0136 - val_accuracy: 0.9823
Epoch 3/10
47795/47795 [==============================] - 317s 7ms/step - loss: 0.0351 - accuracy: 0.9820 - val_loss: 0.0137 - val_accuracy: 0.9805
Epoch 4/10
47795/47795 [==============================] - 317s 7ms/step - loss: 0.0341 - accuracy: 0.9815 - val_loss: 0.0138 - val_accuracy: 0.9775
Epoch 5/10
47795/47795 [==============================] - 317s 7ms/step - loss: 0.0337 - accuracy: 0.9808 - val_loss: 0.0144 - val_accuracy: 0.9730
Epoch 6/10
47795/47795 [==============================] - 316s 7ms/step - loss: 0.0334 - accuracy: 0.9810 - val_loss: 0.0134 - val_accuracy: 0.9801
Epoch 7/10
47795/47795 [==============================] - 317s 7ms/step - loss: 0.0332 - accuracy: 0.9806 - val_

In [ ]:
# Train the model with class weights
model.fit(train_padded, y_train, epochs=10, batch_size=32, validation_data=(test_padded, y_test), class_weight=class_weights)

Epoch 1/10
47795/47795 [==============================] - 348s 7ms/step - loss: 0.3773 - accuracy: 0.9825 - val_loss: 0.1196 - val_accuracy: 0.9828
Epoch 2/10
47795/47795 [==============================] - 308s 6ms/step - loss: 0.3232 - accuracy: 0.9798 - val_loss: 0.1552 - val_accuracy: 0.9786
Epoch 3/10
47795/47795 [==============================] - 307s 6ms/step - loss: 0.3125 - accuracy: 0.9787 - val_loss: 0.1406 - val_accuracy: 0.9744
Epoch 4/10
47795/47795 [==============================] - 308s 6ms/step - loss: 0.3068 - accuracy: 0.9782 - val_loss: 0.1677 - val_accuracy: 0.9741
Epoch 5/10
47795/47795 [==============================] - 308s 6ms/step - loss: 0.3035 - accuracy: 0.9779 - val_loss: 0.1347 - val_accuracy: 0.9773
Epoch 6/10
47795/47795 [==============================] - 309s 6ms/step - loss: 0.3029 - accuracy: 0.9779 - val_loss: 0.1606 - val_accuracy: 0.9752
Epoch 7/10
47795/47795 [==============================] - 310s 6ms/step - loss: 0.3009 - accuracy: 0.9777 - val_

In [ ]:
# Training the model
#model.fit(train_padded, y_train, epochs=10, validation_data=(test_padded, y_test))

In [ ]:
print("Unique labels in test data:", np.unique(y_test))


Unique labels in test data: [0 1]


In [ ]:
y_test.shape

(1529440,)

In [ ]:
from sklearn.metrics import classification_report

# Evaluate the model
predictions = model.predict(test_padded)
predictions = np.round(predictions).flatten()

loss, accuracy = model.evaluate(test_padded, y_test, verbose=0)

# Print the metrics
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Print classification report
report = classification_report(y_test, predictions, target_names=['Class 0', 'Class 1'])
print(report)


11988/11988 [==============================] - 30s 2ms/step
Test Loss: 0.012734010815620422
Test Accuracy: 0.9817001819610596
              precision    recall  f1-score   support

     Class 0       0.98      1.00      0.99    376911
     Class 1       0.41      0.11      0.18      6699

    accuracy                           0.98    383610
   macro avg       0.70      0.56      0.58    383610
weighted avg       0.97      0.98      0.98    383610

